In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'data_preprocessing.ipynb',
 'granular_test',
 'granular_test.zip',
 'SXXP Index_binary.csv']

In [3]:
def parse_date(df):
    """
    This function change strings in first column of each time series to date format
    :return:
    """
    _temp = []
    for d in df:
        try:
            _d = d.split('/')
            _temp.append(dt.datetime(int(_d[2]), int(_d[1]), int(_d[0])))
        except:
            print("Try to parse date format: {}".format(d))
            _temp.append(pd.NaT)

    return _temp

In [4]:
prices_data = pd.read_csv('SXXP Index_binary.csv', low_memory=False)
prices_data

,Unnamed: 0,0876218D LN Equity,Unnamed: 2,Unnamed: 3,1043154Q LN Equity,Unnamed: 5,Unnamed: 6,1294047D SS Equity,Unnamed: 8,Unnamed: 9,...,Unnamed: 3945,ZURN SE Equity,Unnamed: 3947,Unnamed: 3948,ZURN SW Equity,Unnamed: 3950,Unnamed: 3951,SXXP Index,Unnamed: 3953,Unnamed: 3954
0,NaN,Open Price,Closing Price 1 Day Ago,Volume,Open Price,Closing Price 1 Day Ago,Volume,Open Price,Closing Price 1 Day Ago,Volume,...,Volume,Open Price,Closing Price 1 Day Ago,Volume,Open Price,Closing Price 1 Day Ago,Volume,Open Price,Closing Price 1 Day Ago,Volume
1,Dates,PX_OPEN,PX_CLOSE_1D,PX_VOLUME,PX_OPEN,PX_CLOSE_1D,PX_VOLUME,PX_OPEN,PX_CLOSE_1D,PX_VOLUME,...,PX_VOLUME,PX_OPEN,PX_CLOSE_1D,PX_VOLUME,PX_OPEN,PX_CLOSE_1D,PX_VOLUME,PX_OPEN,PX_CLOSE_1D,PX_VOLUME
2,15/04/2019,3.968,4,15044980,184,186,94751,130.6,127.1,27009,...,183453,318.1,317.1,594913,318.1,317.1,594913,387.11,387.53,2040548812
3,12/04/2019,3.968,4,15044980,184,186,94751,130.6,127.1,27009,...,3406127,317.4,316.8,587012,317.4,316.8,587012,386.3,386.91,2245786877
4,11/04/2019,3.968,4,15044980,184,186,94751,130.6,127.1,27009,...,91561,315.4,314.9,462201,315.4,314.9,462201,386.33,386.68,2100927196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,01/01/2002,2.61,2.612,545523,126,125,47161,NaN,NaN,NaN,...,219474,304.71,302.41,206775,304.71,302.41,206775,297.04,297.11,808895400
4512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
to_drop = [name for name in prices_data.columns if name.find('Unnamed') >= 0]
instrument_names = prices_data.drop(to_drop, axis=1).columns.values
len(instrument_names)
prices_data.columns = prices_data.loc[1].values
prices_data.drop([0, 1], axis=0, inplace=True)
prices_data['Dates'] = parse_date(prices_data['Dates'])
prices_data.set_index('Dates', inplace=True)

Try to parse date format: nan
Try to parse date format: nan
Try to parse date format: nan
Try to parse date format: nan


In [6]:
prices_data.drop(pd.NaT, axis=0, inplace=True)

In [7]:
l3 = len(prices_data.columns) // 3
row_nb = len(prices_data)
print(l3)
print(len(instrument_names))

1318
1318


In [8]:
prices_data.columns = np.array([[c+'.'] * 3 for c in instrument_names]).reshape(-1,) + prices_data.columns.values
print(prices_data.columns)

Index(['0876218D LN Equity.PX_OPEN', '0876218D LN Equity.PX_CLOSE_1D',
       '0876218D LN Equity.PX_VOLUME', '1043154Q LN Equity.PX_OPEN',
       '1043154Q LN Equity.PX_CLOSE_1D', '1043154Q LN Equity.PX_VOLUME',
       '1294047D SS Equity.PX_OPEN', '1294047D SS Equity.PX_CLOSE_1D',
       '1294047D SS Equity.PX_VOLUME', '1340426D GF Equity.PX_OPEN',
       ...
       'ZOT SQ Equity.PX_VOLUME', 'ZURN SE Equity.PX_OPEN',
       'ZURN SE Equity.PX_CLOSE_1D', 'ZURN SE Equity.PX_VOLUME',
       'ZURN SW Equity.PX_OPEN', 'ZURN SW Equity.PX_CLOSE_1D',
       'ZURN SW Equity.PX_VOLUME', 'SXXP Index.PX_OPEN',
       'SXXP Index.PX_CLOSE_1D', 'SXXP Index.PX_VOLUME'],
      dtype='object', length=3954)


In [69]:
data_raw = pd.DataFrame(data=np.hstack([prices_data.iloc[:, 0: 3].values,
                                        prices_data.iloc[:, 0:1].shift(-1, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, 1:2].shift(-1, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, 0:1].shift(-10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, 1:2].shift(-10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, 0:1].shift(10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, 0:1].shift(1, axis=0, fill_value=np.NaN).values,
                                        np.array([instrument_names[0]] * row_nb).reshape(-1, 1)]),
                                      columns=['Open', 'Close1', 'Volume', 'Open1', 'Close2', 'Open10', 'Close11', 'OpenNext10', 'OpenNext1','ticker'], 
                        index=prices_data.index)
for i in range(1, l3):
    row = np.hstack([prices_data.iloc[:, i * 3: (i + 1) * 3].values,
                                        prices_data.iloc[:, i * 3:i * 3 + 1].shift(-1, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, i * 3 + 1:i * 3 + 2].shift(-1, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, i * 3:i * 3 + 1].shift(-10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, i * 3 + 1:i * 3 + 2].shift(-10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, i * 3:i * 3 + 1].shift(10, axis=0, fill_value=np.NaN).values,
                                        prices_data.iloc[:, i * 3:i * 3 + 1].shift(1, axis=0, fill_value=np.NaN).values,
                                        np.array([instrument_names[i]] * row_nb).reshape(-1, 1)])
    data_raw = data_raw.append(pd.DataFrame(data=row, 
                                            columns=['Open', 'Close1', 'Volume', 'Open1', 'Close2', 'Open10', 'Close11', 'OpenNext10', 'OpenNext1', 'ticker'], 
                                            index=prices_data.index))

In [70]:
for col in data_raw.columns[:-1]:
    data_raw[col] = data_raw[col].astype('float')

In [71]:
data_raw.reset_index(inplace=True)
data_raw.set_index(['Dates', 'ticker'], inplace=True)
data_raw = data_raw.replace(0, np.NaN).ffill().bfill()

In [72]:
data_raw['returnRawOpen1'] = np.log(data_raw['Open'].values / data_raw['Open1'].values)
data_raw['returnRawOpen10'] = np.log(data_raw['Open'].values / data_raw['Open10'].values)
data_raw['returnRawClose1'] = np.log(data_raw['Close1'].values / data_raw['Close2'].values)
data_raw['returnRawClose10'] = np.log(data_raw['Close1'].values / data_raw['Close11'].values)
data_raw['returnsOpenNextMktres10'] = np.log(data_raw['OpenNext10'].values / data_raw['Open'].values)
data_raw['returnsOpenNextMktres1'] = np.log(data_raw['OpenNext1'].values / data_raw['Open'].values)

In [76]:
data_raw.head()

,,Open,Close1,Volume,Open1,Close2,Open10,Close11,OpenNext10,OpenNext1,returnRawOpen1,returnRawOpen10,returnRawClose1,returnRawClose10,returnsOpenNextMktres10,returnsOpenNextMktres1
Dates,ticker,,,,,,,,,,,,,,,
2019-04-15,0876218D LN Equity,3.968,4.0,15044980.0,3.968,4.0,3.968,4.0,3.968,3.968,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-12,0876218D LN Equity,3.968,4.0,15044980.0,3.968,4.0,3.968,4.0,3.968,3.968,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-11,0876218D LN Equity,3.968,4.0,15044980.0,3.968,4.0,3.968,4.0,3.968,3.968,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-10,0876218D LN Equity,3.968,4.0,15044980.0,3.968,4.0,3.968,4.0,3.968,3.968,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-09,0876218D LN Equity,3.968,4.0,15044980.0,3.968,4.0,3.968,4.0,3.968,3.968,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
data_raw.to_csv('sxxp_data.csv')